### import dataset

read csv file using pandas and get rid of unnamed index column

In [67]:
import pandas as pd

In [68]:
df = pd.read_csv("./dataset/WomensClothingE-CommerceReviews.csv")

In [69]:
df.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [70]:
df = df.drop(df.columns[0], axis='columns')

In [71]:
pd.DataFrame.info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 10 columns):
Clothing ID                23486 non-null int64
Age                        23486 non-null int64
Title                      19676 non-null object
Review Text                22641 non-null object
Rating                     23486 non-null int64
Recommended IND            23486 non-null int64
Positive Feedback Count    23486 non-null int64
Division Name              23472 non-null object
Department Name            23472 non-null object
Class Name                 23472 non-null object
dtypes: int64(5), object(5)
memory usage: 1.8+ MB


fill na with empty string instead of drop the rows.

In [89]:
df.fillna('', inplace=True)

### preprocess strings of "review text"

In [90]:
reviewDf = df[["Clothing ID", "Review Text"]]

In [91]:
reviewDf.head()

,Clothing ID,Review Text
0,767,Absolutely wonderful - silky and sexy and comf...
1,1080,Love this dress! it's sooo pretty. i happene...
2,1077,I had such high hopes for this dress and reall...
3,1049,"I love, love, love this jumpsuit. it's fun, fl..."
4,847,This shirt is very flattering to all due to th...


In [92]:
pd.DataFrame.info(reviewDf)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 2 columns):
Clothing ID    23486 non-null int64
Review Text    23486 non-null object
dtypes: int64(1), object(1)
memory usage: 367.0+ KB


remove stopwords, tokenizing

In [93]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [94]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sieunbae/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [95]:
stopword = set(stopwords.words('english'))

In [98]:
def preprocessing(data):
    data = data.lower()
    words = RegexpTokenizer(r'[a-z]+').tokenize(data)
    words = [w for w in words if not w in stopword]
    
    for pos in [wordnet.NOUN, wordnet.VERB, wordnet.ADJ, wordnet.ADV]:
        words = [WordNetLemmatizer().lemmatize(x, pos) for x in words]
    
    return " ".join(words)

In [99]:
reviewDf['New Text'] = reviewDf["Review Text"].map(preprocessing)

/Users/sieunbae/opt/anaconda3/envs/appliedNLP/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### SentiWordNet

In [100]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/sieunbae/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [101]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [102]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def swn_polarity(text):
    sentiment = 0.0
    

#### Using 'rank' feature

In [103]:
def rank_polarity(row):
    if row['Rating'] >= 4:
        return 'Positive'
    elif row['Rating'] >= 3:
        return 'Neutral'
    else:
        return 'Negative'

In [104]:
reviewDf['Rank_Class'] = df['Rating'].map(rank_polarity)

TypeError: 'int' object is not subscriptable